In [1]:
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
UNI = "cld2167"

In [2]:
def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip().split("-")[1] for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0

    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [4]:
# Load the data
data_root_path = 'cifar10-hw1/'
X_train_total, y_train_total = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [5]:
# X_train=X_train_total[:,:45000]
# X_val=X_train_total[:,45000:]
# y_train=y_train_total[:45000]
# y_val=y_train_total[45000:]

## probably best to train on EVERYTHING


X_train=X_train_total[:,:45000]
X_val=X_train_total[:,45000:]
y_train=y_train_total[:45000]
y_val=y_train_total[45000:]

In [6]:
def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot
print(y_train.shape)
y_train=one_hot(y_train)
y_val=one_hot(y_val)
print(y_train.shape)

(45000,)
(10, 45000)


In [7]:
def get_batch(X_train,y_train,batch_size=100):
    ix = np.random.choice(X_train.shape[1], batch_size, replace=True)
    X_train_batch=X_train[:,ix]
    y_train_batch=y_train[:,ix]
    return X_train_batch.T,y_train_batch.T

In [8]:
X_train.shape

(3072, 45000)

In [9]:
def weight_variable(shape, name):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name=name)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [10]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [11]:
###### first conv layer


CONV_1_DEPTH = 32
CONV_1_SIZE = 7
CONV_2_DEPTH = 32
CONV_2_SIZE = 5
CONV_3_DEPTH = 16
CONV_3_SIZE = 5
CONV_4_DEPTH = 8
CONV_4_SIZE = 3
CONV_5_DEPTH = 8
CONV_5_SIZE = 3
CONV_6_DEPTH = 8
CONV_6_SIZE = 3
CONV_7_DEPTH = 8
CONV_7_SIZE = 3
CONV_8_DEPTH = 8
CONV_8_SIZE = 3

epsilon = 1e-3
FULLY_CONNECTED_SIZE = 1024

keep_prob = tf.placeholder(tf.float32)

x = tf.placeholder(tf.float32, shape=[None, 3072])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W_conv1 = weight_variable([CONV_1_SIZE, CONV_1_SIZE, 3, CONV_1_DEPTH], name="conv_1_weights")
b_conv1 = bias_variable([CONV_1_DEPTH])
#x_image = tf.reshape(x, [-1, 28, 28, 1])
tf.summary.histogram("W_conv1", W_conv1)


x_image = tf.reshape(x, [-1, 32, 32, 3])
conv_1 = conv2d(x_image, W_conv1) + b_conv1

batch_mean1, batch_var1 = tf.nn.moments(conv_1,[0])
scale1 = tf.Variable(tf.ones([CONV_1_DEPTH]))
beta1 = tf.Variable(tf.zeros([CONV_1_DEPTH]))
batch1 = tf.nn.batch_normalization(conv_1,batch_mean1,batch_var1,beta1,scale1,epsilon)

h_conv1 = tf.nn.relu(batch1)
h_pool1 = max_pool_2x2(h_conv1)
h_drop1 = tf.nn.dropout(h_pool1, keep_prob)

#second conv layer
W_conv2 = weight_variable([CONV_2_SIZE, CONV_2_SIZE, CONV_1_DEPTH, CONV_2_DEPTH], name="conv_2_weights")
b_conv2 = bias_variable([CONV_2_DEPTH])
conv_2 = conv2d(h_drop1, W_conv2) + b_conv2
tf.summary.histogram("W_conv2", W_conv2)

batch_mean2, batch_var2 = tf.nn.moments(conv_2,[0])
scale2 = tf.Variable(tf.ones([CONV_2_DEPTH]))
beta2 = tf.Variable(tf.zeros([CONV_2_DEPTH]))
batch2 = tf.nn.batch_normalization(conv_2,batch_mean2,batch_var2,beta2,scale2,epsilon)

h_conv2 = tf.nn.relu(batch2)
h_pool2 = max_pool_2x2(h_conv2)
h_drop2 = tf.nn.dropout(h_pool2, keep_prob)

#third conv layer
W_conv3 = weight_variable([CONV_3_SIZE, CONV_3_SIZE, CONV_2_DEPTH, CONV_3_DEPTH], name="conv_3_weights")
b_conv3 = bias_variable([CONV_3_DEPTH])
conv_3 = conv2d(h_drop2, W_conv3) + b_conv3
tf.summary.histogram("W_conv3", W_conv3)

batch_mean3, batch_var3 = tf.nn.moments(conv_3,[0])
scale3 = tf.Variable(tf.ones([CONV_3_DEPTH]))
beta3 = tf.Variable(tf.zeros([CONV_3_DEPTH]))
batch3 = tf.nn.batch_normalization(conv_3,batch_mean3,batch_var3,beta3,scale3,epsilon)

h_conv3 = tf.nn.relu(batch3)
h_pool3 = max_pool_2x2(h_conv3)
h_drop3 = tf.nn.dropout(h_pool3, keep_prob)

#fourth conv layer
W_conv4 = weight_variable([CONV_4_SIZE, CONV_4_SIZE, CONV_3_DEPTH, CONV_4_DEPTH], name="conv_4_weights")
b_conv4 = bias_variable([CONV_4_DEPTH])
conv_4 = conv2d(h_drop3, W_conv4) + b_conv4
tf.summary.histogram("W_conv4_gradients", tf.gradients(W_conv4))

batch_mean4, batch_var4 = tf.nn.moments(conv_4,[0])
scale4 = tf.Variable(tf.ones([CONV_4_DEPTH]))
beta4 = tf.Variable(tf.zeros([CONV_4_DEPTH]))
batch4 = tf.nn.batch_normalization(conv_4,batch_mean4,batch_var4,beta4,scale4,epsilon)

h_conv4 = tf.nn.relu(batch4)
h_pool4 = max_pool_2x2(h_conv4)
h_drop4 = tf.nn.dropout(h_pool4, keep_prob)

# #fifth conv layer
# W_conv5 = weight_variable([CONV_5_SIZE, CONV_5_SIZE, CONV_4_DEPTH, CONV_5_DEPTH], name="conv_5_weights")
# b_conv5 = bias_variable([CONV_5_DEPTH])
# conv_5 = conv2d(h_drop4, W_conv5) + b_conv5

# batch_mean5, batch_var5 = tf.nn.moments(conv_5,[0])
# scale5 = tf.Variable(tf.ones([CONV_5_DEPTH]))
# beta5 = tf.Variable(tf.zeros([CONV_5_DEPTH]))
# batch5 = tf.nn.batch_normalization(conv_5,batch_mean5,batch_var5,beta5,scale5,epsilon)

# h_conv5 = tf.nn.relu(batch5)
# h_pool5 = max_pool_2x2(h_conv5)
# h_drop5 = tf.nn.dropout(h_pool5, keep_prob)

# #sixth conv layer
# W_conv6 = weight_variable([CONV_6_SIZE, CONV_6_SIZE, CONV_5_DEPTH, CONV_6_DEPTH], name="conv_6_weights")
# b_conv6 = bias_variable([CONV_6_DEPTH])
# conv_6 = conv2d(h_drop5, W_conv6) + b_conv6

# batch_mean6, batch_var6 = tf.nn.moments(conv_6,[0])
# scale6 = tf.Variable(tf.ones([CONV_6_DEPTH]))
# beta6 = tf.Variable(tf.zeros([CONV_6_DEPTH]))
# batch6 = tf.nn.batch_normalization(conv_6,batch_mean6,batch_var6,beta6,scale6,epsilon)

# h_conv6 = tf.nn.relu(batch6)
# h_pool6 = max_pool_2x2(h_conv6)
# h_drop6 = tf.nn.dropout(h_pool6, keep_prob)
# #seventh conv layer
# W_conv7 = weight_variable([CONV_7_SIZE, CONV_7_SIZE, CONV_6_DEPTH, CONV_7_DEPTH], name="conv_7_weights")
# b_conv7 = bias_variable([CONV_7_DEPTH])
# conv_7 = conv2d(h_pool6, W_conv7) + b_conv7

# batch_mean7, batch_var7 = tf.nn.moments(conv_7,[0])
# scale7 = tf.Variable(tf.ones([CONV_7_DEPTH]))
# beta7 = tf.Variable(tf.zeros([CONV_7_DEPTH]))
# batch7 = tf.nn.batch_normalization(conv_7,batch_mean7,batch_var7,beta7,scale7,epsilon)

# h_conv7 = tf.nn.relu(batch7)
# h_pool7 = max_pool_2x2(h_conv7)

# #eigth conv layer
# W_conv8 = weight_variable([CONV_8_SIZE, CONV_8_SIZE, CONV_7_DEPTH, CONV_8_DEPTH], name="conv_8_weights")
# b_conv8 = bias_variable([CONV_8_DEPTH])
# conv_8 = conv2d(h_pool7, W_conv8) + b_conv8

# batch_mean8, batch_var8 = tf.nn.moments(conv_8,[0])
# scale8 = tf.Variable(tf.ones([CONV_8_DEPTH]))
# beta8 = tf.Variable(tf.zeros([CONV_8_DEPTH]))
# batch8 = tf.nn.batch_normalization(conv_8,batch_mean8,batch_var8,beta8,scale8,epsilon)

# h_conv8 = tf.nn.relu(batch8)
# h_pool8 = max_pool_2x2(h_conv8)

#dense layer
W_fc1 = weight_variable([2 * 2 * CONV_4_DEPTH, FULLY_CONNECTED_SIZE], name="first_fc")
b_fc1 = bias_variable([FULLY_CONNECTED_SIZE])

h_pool8_flat = tf.reshape(h_drop4, [-1, 2*2*CONV_4_DEPTH])
h_fc1 = tf.nn.relu(tf.matmul(h_pool8_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


#readout
W_fc2 = weight_variable([FULLY_CONNECTED_SIZE, 10], name="last_fc")
tf.summary.histogram("last_fc", W_fc2)
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

beta = 0.01
# calculate both softmax and softmax cross entropy, one for training one for testing
softmax = tf.nn.softmax(y_conv)
softmax_ce = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv)

cross_entropy = tf.reduce_mean(softmax_ce)

# l2 regularization
regularizer = tf.nn.l2_loss(W_fc2) + tf.nn.l2_loss(W_fc1)
loss = cross_entropy + beta * regularizer
#train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
train_step = tf.train.GradientDescentOptimizer(.5).minimize(loss)


# summaries
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar("loss", loss)
tf.summary.scalar("accuracy", accuracy)
summaries = tf.summary.merge_all()

# merge all summaries into a single "operation" which we can execute in a session 
#summary_op = tf.merge_all_summaries()

with tf.Session() as sess:
    #sess = tf_debug.LocalCLIDebugWrapperSession(sess)
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter("output", sess.graph)

    for i in range(2501):
        batch_x,batch_y = get_batch(X_train,y_train,batch_size=500)
        #train_step.run(feed_dict={x: batch_x, y_: batch_y})
        
        _, summary  = sess.run([ train_step, summaries], feed_dict={x: batch_x, y_: batch_y, keep_prob:1.0})
        
        # write log
        if i % 250 == 0:
            writer.add_summary(summary, i)
            #train_accuracy = accuracy.eval(feed_dict={x: batch_x, y_: batch_y})
            print('train accuracy %g' % accuracy.eval(feed_dict={x: X_train.T, y_: y_train.T}))
            print('step %d, test accuracy %g' % (i, accuracy.eval(feed_dict={x: X_val.T, y_: y_val.T, keep_prob:1.0})))
            
    # network trained, now make predictions on the test set
    prediction = tf.argmax(y_conv,1)
    preds = softmax.eval(feed_dict={x: X_test.T, keep_prob:1.0})
    print(preds.shape)
    np.save("hw2_ans2_" + UNI + ".npy",preds)
    
print("DONE NO ERROR")

step 0, test accuracy 0.1484
step 100, test accuracy 0.4186
step 200, test accuracy 0.4608
step 300, test accuracy 0.4904
step 400, test accuracy 0.4984
step 500, test accuracy 0.5284
step 600, test accuracy 0.565
step 700, test accuracy 0.5822
step 800, test accuracy 0.5956
step 900, test accuracy 0.545
step 1000, test accuracy 0.6238
step 1100, test accuracy 0.59
step 1200, test accuracy 0.6212
step 1300, test accuracy 0.6252
step 1400, test accuracy 0.631
step 1500, test accuracy 0.6402
step 1600, test accuracy 0.634
step 1700, test accuracy 0.6424
step 1800, test accuracy 0.6412
step 1900, test accuracy 0.6448
step 2000, test accuracy 0.6522
step 2100, test accuracy 0.6478
step 2200, test accuracy 0.6224
step 2300, test accuracy 0.6464
step 2400, test accuracy 0.6614
step 2500, test accuracy 0.6618
(10000, 10)
DONE NO ERROR


In [12]:
preds

array([[  1.96037404e-02,   1.27094099e-02,   1.50957247e-02, ...,
          1.36331525e-02,   8.32180902e-02,   1.10961460e-02],
       [  1.24967340e-02,   9.07877982e-01,   4.45618753e-05, ...,
          1.54284699e-05,   7.03403428e-02,   9.06825718e-03],
       [  1.82488382e-01,   8.25485121e-03,   2.55368594e-02, ...,
          9.26849432e-03,   4.84301239e-01,   1.15367360e-02],
       ..., 
       [  2.23757685e-04,   1.21659803e-04,   1.78992357e-02, ...,
          9.73815098e-03,   1.92559222e-04,   1.84655102e-04],
       [  4.64018807e-02,   7.58669794e-01,   1.40115926e-02, ...,
          3.59270386e-02,   1.01853516e-02,   3.48466933e-02],
       [  3.64966691e-05,   1.01464893e-05,   2.40503432e-04, ...,
          9.54999447e-01,   3.82222743e-06,   1.99362767e-05]], dtype=float32)